In [ ]:
import pandas as pd
import torch
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [ ]:
train_df=pd.read_csv('webNLG2020_train.csv', index_col=[0])
train_df=train_df.iloc[  :35000,:]
train_df=train_df.sample(frac = 1)
batch_size=8
num_of_batches=int(len(train_df)/batch_size)

In [ ]:
num_of_batches

In [ ]:
train_df

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
    print("Running on the GPU")
else:
    dev = torch.device("cpu")
    print("Running on the CPU")

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base',
                                             return_dict=True)
#moving the model to GPU
model.to(dev)

In [ ]:
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [ ]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [ ]:
#Sets the module in training mode
model.train()
num_of_epochs = 3

loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
  
    running_loss=0

    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
        inputbatch=[]
        labelbatch=[]
        new_df=train_df[i*batch_size:i*batch_size+batch_size]
        for indx,row in new_df.iterrows():
            input = 'WebNLG: '+row['input_text']+'</s>' 
            labels = row['target_text']+'</s>'   
            inputbatch.append(input)
            labelbatch.append(labels)
        inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
        labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
        inputbatch=inputbatch.to(dev)
        labelbatch=labelbatch.to(dev)

        # clear out the gradients of all Variables 
        optimizer.zero_grad()

        # Forward propogation
        outputs = model(input_ids=inputbatch, labels=labelbatch)
        loss = outputs.loss
        loss_num=loss.item()
        logits = outputs.logits
        running_loss+=loss_num
        if i%10 ==0:      
            loss_per_10_steps.append(loss_num)
        out.update(progress(loss_num,i, num_of_batches+1))

        # calculating the gradients
        loss.backward()

        #updating the params
        optimizer.step()
    
    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))

In [ ]:
torch.save(model.state_dict(),'trained_model/pytorch_model.bin')

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('trained_model', 
                                                return_dict=True)
import re

def generate(text):
    texts = text.split(".")
    result = ""
    for txt in texts:
        model.eval()
        input_ids = tokenizer.encode("WebNLG:{} </s>".format(txt), 
                                   return_tensors="pt")  
        outputs = model.generate(input_ids)
        result += tokenizer.decode(outputs[0])
    result = re.sub('<pad>|</s>',"",result)
    return result